# Analysis of Mercury Levels in Albertan Fish
## Cleaning Notebook

### Introduction
Mercury contamination in fish is a significant environmental and public health concern. This notebook is dedicated to the data cleaning and preparation process for an in-depth analysis of mercury levels in various fish species across Alberta. The goal is to prepare a dataset that accurately reflects the mercury concentrations in these species, which can then be used for detailed analysis.

### Data Source
[Data was sourced](https://open.alberta.ca/opendata/chemical-monitoring-in-local-foods-mercury-in-fish) from the Government of Alberta’s open data portal, encompassing measurements from 1997 to 2020. The dataset includes standardized mercury testing results from multiple fish species across Alberta's seven distinct regions.

### Data Processing and Cleaning
In this phase, the initial dataset contained 6008 records and 25 columns. To refine the dataset for analysis, irrelevant columns were removed and missing values were addressed. This resulted in a cleaned dataset of 3781 records across 13 columns, spanning from 1997 to 2020.

### Limitations
This process focuses solely on the data available up to 2020. It is important to note that some rows with missing data or 'Unknown' values were excluded (except in the case of the 'Sex' of the species) to maintain the integrity and accuracy of the measurements.

### Citations and References
Relevant literature and supplementary data sources are cited where applicable to provide context and support for the data cleaning process.

## Cleaning Process

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
df = pd.read_excel("../data/hg-in-fish_ah-1997-2020_20230921.xlsx", engine="openpyxl")

In [33]:
print(df.head())
print(df.info())
print(df.describe())

      Program_ID      Sample_ID Collection Date Date_flag Waterbody Name  \
0  13-AK-LKWH-01  14-AK-LKWH-01      2013-09-18         0     Amisk Lake   
1  13-AK-LKWH-02  14-AK-LKWH-02      2013-09-19         0     Amisk Lake   
2  10-AK-NRPK-01     FHg11-0101      2010-09-16         0     Amisk Lake   
3  10-AK-NRPK-02     FHg11-0108      2010-09-16         0     Amisk Lake   
4  10-AK-NRPK-03     FHg11-0112      2010-09-16         0     Amisk Lake   

  Waterbody Type   Latitude   Longitude  Land Use Region FWMIS ID  ...  \
0           Lake  54.597508 -112.636283  Upper Athabasca     3916  ...   
1           Lake  54.597508 -112.636283  Upper Athabasca     3916  ...   
2           Lake  54.597508 -112.636283  Upper Athabasca     3916  ...   
3           Lake  54.597508 -112.636283  Upper Athabasca     3916  ...   
4           Lake  54.597508 -112.636283  Upper Athabasca     3916  ...   

  Fork LengthMeasurement  Total Length (mm) Total LengthMeasurement  \
0                   Field  

In [34]:
df.isnull().sum()  # This prints out how many null values exist in each column.

Program_ID                    1
Sample_ID                     2
Collection Date               0
Date_flag                     1
Waterbody Name                0
Waterbody Type                0
 Latitude                     0
 Longitude                    0
Land Use Region               0
FWMIS ID                      0
Species Code                  0
Common Name                   0
Binomial Name                 0
Sex                           0
Fork Length (mm)              0
Fork LengthMeasurement     1082
Total Length (mm)             0
Total LengthMeasurement    1200
LengthEstimateFactor       6006
Weight (g)                    0
WeightMeasurement           138
Maturity                      0
Age (years)                1587
Aging Structure            1587
Hg (mg/kg)                    0
dtype: int64

I prefer to remove the columns I don't need first, then start addressing the rest of the cleaning/processing steps.

In [35]:
drop_columns = [
    "Program_ID", # Doesn't contribute to analysis objectives.
    "Sample_ID", # Doesn't contribute to analysis objectives.
    "Date_flag", # Doesn't contribute to analysis objectives.
    "FWMIS ID", # Doesn't contribute to analysis objectives.
    "Binomial Name", # Latin (scientific) name of the fish species redundant with Common Name.
    "LengthEstimateFactor", # Not useable due to high % of missing data.
    "Total LengthMeasurement", # Location of measurement, not relevant.
    "WeightMeasurement", # Location of measurement, not relevant.
    "Maturity", # Doesn't contribute to analysis objectives.
    "Age (years)", # Doesn't contribute to analysis objectives.
    "Aging Structure", # Doesn't contribute to analysis objectives.
]

In [36]:
df_slim = df.copy()  # Makes a copy of the original df for me to make my changes to

In [37]:
df_slim = df_slim.drop(columns=drop_columns)  # Drops the columns I don't need

In [38]:
# Drop column 'Fork LengthMeasurement ' from df_slim 
df_slim.drop("Fork LengthMeasurement ", axis=1, inplace=True)

In [39]:
df_slim.info()
df_slim.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6008 entries, 0 to 6007
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Collection Date    6008 non-null   datetime64[ns]
 1   Waterbody Name     6008 non-null   object        
 2   Waterbody Type     6008 non-null   object        
 3    Latitude          6008 non-null   object        
 4    Longitude         6008 non-null   object        
 5   Land Use Region    6008 non-null   object        
 6   Species Code       6008 non-null   object        
 7   Common Name        6008 non-null   object        
 8   Sex                6008 non-null   object        
 9   Fork Length (mm)   6008 non-null   object        
 10  Total Length (mm)  6008 non-null   object        
 11  Weight (g)         6008 non-null   object        
 12  Hg (mg/kg)         6008 non-null   object        
dtypes: datetime64[ns](1), object(12)
memory usage: 610.3+ KB


Index(['Collection Date', 'Waterbody Name', 'Waterbody Type', ' Latitude',
       ' Longitude', 'Land Use Region', 'Species Code', 'Common Name', 'Sex',
       'Fork Length (mm)', 'Total Length (mm)', 'Weight (g)', 'Hg (mg/kg)'],
      dtype='object')

In [40]:
# To check the unique values in each qualitative column

categorical_columns = [
    "Waterbody Name",
    "Waterbody Type",
    "Land Use Region",
    "Species Code",
    "Common Name",
    "Sex",
]
for col in categorical_columns:
    df_slim[col] = df_slim[col].astype("category")
    # Optionally, check for unique values
    print(f"Unique values in {col}: {df_slim[col].unique()}" + "\n")

Unique values in Waterbody Name: ['Amisk Lake', 'Angling Lake', 'Arm Lake', 'Athabasca River (between Hinton and Whitecourt)', 'Baptiste Lake', ..., 'Willow Creek', 'Willow Lake (Gregoire Lake)', 'Winagami Lake', 'Wizard Lake', 'Wolf Lake']
Length: 137
Categories (137, object): ['Amisk Lake', 'Angling Lake', 'Arm Lake', 'Athabasca River (between Hinton and Whitecourt)', ..., 'Willow Lake (Gregoire Lake)', 'Winagami Lake', 'Wizard Lake', 'Wolf Lake']

Unique values in Waterbody Type: ['Lake', 'River', 'Reservoir', 'Canal', 'Stormwater Pond', 'Lake ']
Categories (6, object): ['Canal', 'Lake', 'Lake ', 'Reservoir', 'River', 'Stormwater Pond']

Unique values in Land Use Region: ['Upper Athabasca', 'Lower Athabasca', 'North Saskatchewan', 'Red Deer', 'Lower Peace', 'South Saskatchewan', 'Upper Peace']
Categories (7, object): ['Lower Athabasca', 'Lower Peace', 'North Saskatchewan', 'Red Deer', 'South Saskatchewan', 'Upper Athabasca', 'Upper Peace']

Unique values in Species Code: ['LKWH', 'N

After viewing the unique values each column, there are obvious duplicates in some of the columns.

1. For Waterbody type, "Lake" appears twice, due to an extra whitespace in "Lake ".

2. For the Sex column, there are 5 unique values when there should be only 2 (Male and Female).

3. There is also a discrepancy between the unique values in Species Code and Common Name, as there should be the exact same number of unique values in both columns.

In [41]:
df_slim["Sex"] = df_slim[
    "Sex"
].str.strip()  # Removes the whitespace from the column variables
df_slim["Waterbody Name"] = df_slim[
    "Waterbody Name"
].str.strip()  # Checking Waterbody Name for any whitespaces
df_slim["Waterbody Type"] = df_slim[
    "Waterbody Type"
].str.strip()  # Removing any whitespaces, specifically from the two versions of "Lake"
df_slim["Species Code"] = df_slim[
    "Species Code"
].str.strip()  # Checking Species Code for any whitespaces
df_slim["Common Name"] = df_slim[
    "Common Name"
].str.strip()  # Issue here is ' Brook Trout' is not 'Brook Trout'.

In [42]:
print(df_slim["Sex"].unique())

['Male' 'Female' 'Unknown']


In [43]:
print(
    df_slim["Common Name"].unique()
)  

['Lake Whitefish' 'Northern Pike' 'Walleye' 'Mountain Whitefish'
 'Yellow Perch' 'Brown Trout' 'Longnose Sucker' 'Rainbow Trout'
 'White Sucker' 'Burbot' 'Brook Trout' 'Lake Trout' 'Cisco' 'Goldeye'
 'Mooneye' 'Sauger' 'Flathead Chub' 'Shorthead Redhorse' 'Unknown'
 'Cutthroat Trout' 'Lake Chub (Cisco)' 'Longnose Dace' 'Trout Perch']


### From the earlier unique value check:

1. 137 Unique values in Waterbody Name: ['Amisk Lake', 'Angling Lake', 'Arm Lake', 'Athabasca River (between Hinton and Whitecourt)', 'Baptiste Lake', ..., 'Willow Creek', 'Willow Lake (Gregoire Lake)', 'Winagami Lake', 'Wizard Lake', 'Wolf Lake']

2. 6 Unique values in Waterbody Type: ['Lake', 'River', 'Reservoir', 'Canal', 'Stormwater Pond', 'Lake ']

3. 7 Unique values in Land Use Region: ['Upper Athabasca', 'Lower Athabasca', 'North Saskatchewan', 'Red Deer', 'Lower Peace', 'South Saskatchewan', 'Upper Peace']

4. 23 Unique values in Species Code: ['LKWH', 'NRPK', 'WALL', 'MNWH', 'YLPR', ..., 'Unknown', 'CTTR', 'LKCH', 'LNDC', 'TRPR']

5. 24 Unique values in Common Name: ['Lake Whitefish', 'Northern Pike', 'Walleye', 'Mountain Whitefish', 'Yellow Perch', ..., 'Unknown', 'Cutthroat Trout', 'Lake Chub (Cisco)', 'Longnose Dace', 'Trout Perch']

6. 5 Unique values in Sex: ['Male', 'Female', 'Unknown', 'Female ', 'Male ']


In [44]:
# Checking the unique variables again, after removing whitespaces:
for col in categorical_columns:
    df_slim[col] = df_slim[col].astype("category")
    # Optionally, check for unique values
    print(f"Unique values in {col}: {df_slim[col].unique()}" + "\n")

Unique values in Waterbody Name: ['Amisk Lake', 'Angling Lake', 'Arm Lake', 'Athabasca River (between Hinton and Whitecourt)', 'Baptiste Lake', ..., 'Willow Creek', 'Willow Lake (Gregoire Lake)', 'Winagami Lake', 'Wizard Lake', 'Wolf Lake']
Length: 136
Categories (136, object): ['Amisk Lake', 'Angling Lake', 'Arm Lake', 'Athabasca River (between Hinton and Whitecourt)', ..., 'Willow Lake (Gregoire Lake)', 'Winagami Lake', 'Wizard Lake', 'Wolf Lake']

Unique values in Waterbody Type: ['Lake', 'River', 'Reservoir', 'Canal', 'Stormwater Pond']
Categories (5, object): ['Canal', 'Lake', 'Reservoir', 'River', 'Stormwater Pond']

Unique values in Land Use Region: ['Upper Athabasca', 'Lower Athabasca', 'North Saskatchewan', 'Red Deer', 'Lower Peace', 'South Saskatchewan', 'Upper Peace']
Categories (7, object): ['Lower Athabasca', 'Lower Peace', 'North Saskatchewan', 'Red Deer', 'South Saskatchewan', 'Upper Athabasca', 'Upper Peace']

Unique values in Species Code: ['LKWH', 'NRPK', 'WALL', 'MNW

#### Breakdown of Cleaning Whitespace from Columns:

Unique values in Waterbody Name (Before -> After): 137 -> 136

Unique values in Waterbody Type (Before -> After): 6 -> 5

Unique values in Land Use Region (Before -> After): 7 -> 7

Unique values in Species Code (Before -> After): 23 -> 23

Unique values in Common Name (Before -> After): 24 -> 23

Unique values in Sex (Before -> After): 5 -> 3

In [45]:
df_slim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6008 entries, 0 to 6007
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Collection Date    6008 non-null   datetime64[ns]
 1   Waterbody Name     6008 non-null   category      
 2   Waterbody Type     6008 non-null   category      
 3    Latitude          6008 non-null   object        
 4    Longitude         6008 non-null   object        
 5   Land Use Region    6008 non-null   category      
 6   Species Code       6008 non-null   category      
 7   Common Name        6008 non-null   category      
 8   Sex                6008 non-null   category      
 9   Fork Length (mm)   6008 non-null   object        
 10  Total Length (mm)  6008 non-null   object        
 11  Weight (g)         6008 non-null   object        
 12  Hg (mg/kg)         6008 non-null   object        
dtypes: category(6), datetime64[ns](1), object(6)
memory usage: 377.

In [46]:
# Format the "Collection Date" column to consistant date format
# df_slim["Collection Date"] = pd.to_datetime(df_slim["Collection Date"]).dt.strftime(
#     "%m/%d/%Y"
# )

# Convert 'Collection Date' column to datetime64 - For purpose of time series analysis
df_slim["Collection Date"] = pd.to_datetime(df_slim["Collection Date"])

I converted the Datetime column to string format, to ensure the date was consistent, but then converted it to datetime64 format to allow for easier manipulation and analysis of the data (if I decide to do so).

I realized that I need to convert the quantitative columns to numeric format, as they were originally in an object format, but the Unknown values need to be dealt with first.

In [47]:
df_slim.columns

Index(['Collection Date', 'Waterbody Name', 'Waterbody Type', ' Latitude',
       ' Longitude', 'Land Use Region', 'Species Code', 'Common Name', 'Sex',
       'Fork Length (mm)', 'Total Length (mm)', 'Weight (g)', 'Hg (mg/kg)'],
      dtype='object')

In [48]:
df_slim.columns = df_slim.columns.str.strip() # Latitude, Longitude column names contain whitespace

In [49]:
# Changing Unknown values to NaN, then changing column datatype to float

df_slim["Fork Length (mm)"] = df_slim["Fork Length (mm)"].replace("Unknown", np.nan)
df_slim["Fork Length (mm)"] = df_slim["Fork Length (mm)"].astype(float)

In [50]:
df_slim['Total Length (mm)'] = df_slim['Total Length (mm)'].replace('Unknown', np.nan)
df_slim['Total Length (mm)'] = df_slim['Total Length (mm)'].astype(float)

In [51]:
df_slim['Weight (g)'] = df_slim['Weight (g)'].replace('Unknown', np.nan)
df_slim['Weight (g)'] = df_slim['Weight (g)'].astype(float)

In [52]:
# Don't need the Collection Date to include time
df_slim["Collection Date"] = pd.to_datetime(df_slim["Collection Date"]).dt.date

In [60]:
# Drop rows with missing data across all columns
df_slim = df_slim.dropna()
# Drop the rows that are missing values in the Hg (mg/kg) column - Noticed this had 2 rows with missing mercury data. Now all of the columns have no missing values.
df_slim = df_slim.dropna(subset=["Hg (mg/kg)"])

In [61]:
df_slim.info()
df_slim.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 3781 entries, 0 to 6007
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Collection Date    3781 non-null   object  
 1   Waterbody Name     3781 non-null   category
 2   Waterbody Type     3781 non-null   category
 3   Latitude           3781 non-null   object  
 4   Longitude          3781 non-null   object  
 5   Land Use Region    3781 non-null   category
 6   Species Code       3781 non-null   category
 7   Common Name        3781 non-null   category
 8   Sex                3781 non-null   category
 9   Fork Length (mm)   3781 non-null   float64 
 10  Total Length (mm)  3781 non-null   float64 
 11  Weight (g)         3781 non-null   float64 
 12  Hg (mg/kg)         3781 non-null   float64 
dtypes: category(6), float64(4), object(3)
memory usage: 269.4+ KB


,Fork Length (mm),Total Length (mm),Weight (g),Hg (mg/kg)
count,3781.000000,3781.000000,3781.000000,3781.000000
mean,513.544036,549.151812,1547.959535,0.347501
std,133.196090,138.646409,1161.504140,0.303631
min,130.000000,139.000000,25.000000,0.007000
25%,425.000000,459.000000,827.000000,0.118000
50%,500.000000,538.000000,1250.000000,0.264000
75%,589.000000,626.000000,1910.000000,0.481000
max,1105.000000,1168.000000,10550.000000,2.552000


In [62]:
df_slim["Hg (mg/kg)"] = df_slim['Hg (mg/kg)'].replace("Unknown", np.nan)
df_slim["Hg (mg/kg)"] = df_slim["Hg (mg/kg)"].astype(float)

In [63]:
df_slim.info()
df_slim.describe()



<class 'pandas.core.frame.DataFrame'>
Index: 3781 entries, 0 to 6007
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Collection Date    3781 non-null   object  
 1   Waterbody Name     3781 non-null   category
 2   Waterbody Type     3781 non-null   category
 3   Latitude           3781 non-null   object  
 4   Longitude          3781 non-null   object  
 5   Land Use Region    3781 non-null   category
 6   Species Code       3781 non-null   category
 7   Common Name        3781 non-null   category
 8   Sex                3781 non-null   category
 9   Fork Length (mm)   3781 non-null   float64 
 10  Total Length (mm)  3781 non-null   float64 
 11  Weight (g)         3781 non-null   float64 
 12  Hg (mg/kg)         3781 non-null   float64 
dtypes: category(6), float64(4), object(3)
memory usage: 269.4+ KB


,Fork Length (mm),Total Length (mm),Weight (g),Hg (mg/kg)
count,3781.000000,3781.000000,3781.000000,3781.000000
mean,513.544036,549.151812,1547.959535,0.347501
std,133.196090,138.646409,1161.504140,0.303631
min,130.000000,139.000000,25.000000,0.007000
25%,425.000000,459.000000,827.000000,0.118000
50%,500.000000,538.000000,1250.000000,0.264000
75%,589.000000,626.000000,1910.000000,0.481000
max,1105.000000,1168.000000,10550.000000,2.552000


In [57]:
df_clean = df_slim.copy() # Copy the cleaned df_slim dataframe to df_clean
# Create a csv file of the df_clean df which will be used for analysis in another Jupyter notebook
df_clean.to_csv("df_clean.csv", index=False)

### Final Cleaned Dataset Overview

The final cleaned dataset contains 3781 entries across 13 columns:

- **Collection Date** - Date when the fish sample was collected.
- **Waterbody Name** - Name of the waterbody where the sample was taken.
- **Waterbody Type** - Type of waterbody, e.g., Lake, River, Reservoir.
- **Latitude** - Latitude where the sample was collected.
- **Longitude** - Longitude where the sample was collected.
- **Land Use Region** - Alberta's regional location of each waterbody.
- **Species Code** - A unique four-letter code for each fish species.
- **Common Name** - The common name of the fish.
- **Sex** - Sex of the fish: Male, Female, or Unknown.
- **Fork Length (mm)** - Fish length from nose to tail fork.
- **Total Length (mm)** - Fish length from nose to tail tip.
- **Weight (g)** - Weight of the fish.
- **Hg (mg/kg)** - Total mercury levels in the fish muscle.

*Note: For more detailed descriptions of these data fields, please refer to the [Data Dictionary](link_to_data_dictionary).*

#### This cleaned dataset will be used in the Analysis Notebook, found in this repository.